# Prepare protein structure dataset

- Parse the PDB files into json documents
- Parse the CATH labels from RCSB API, merge into the json documents
- Create index on PDB ID
- Insert json documents into DocumentDB

In [ ]:
stack_name = 'gnn-proteins' # name of CloudFormation stack

In [1]:
import os
import json

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn')
import seaborn as sns
%matplotlib inline

In [2]:
!mkdir -p data

## Download data from [AlphaFold](https://alphafold.ebi.ac.uk/download)

In [3]:
!wget -O data/UP000000805_243232_METJA.tar \
    https://ftp.ebi.ac.uk/pub/databases/alphafold/UP000000805_243232_METJA.tar 

--2021-09-08 20:20:01--  https://ftp.ebi.ac.uk/pub/databases/alphafold/UP000000805_243232_METJA.tar
Resolving ftp.ebi.ac.uk (ftp.ebi.ac.uk)... 193.62.197.74
Connecting to ftp.ebi.ac.uk (ftp.ebi.ac.uk)|193.62.197.74|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 178278400 (170M) [application/octet-stream]
Saving to: ‘data/UP000000805_243232_METJA.tar’

data/UP000000805_24 100%[===================>] 170.02M  10.8MB/s    in 14s     

2021-09-08 20:20:16 (12.2 MB/s) - ‘data/UP000000805_243232_METJA.tar’ saved [178278400/178278400]



In [4]:
!cd data && tar -xvf UP000000805_243232_METJA.tar

AF-O06917-F1-model_v1.cif.gz
AF-O06917-F1-model_v1.pdb.gz
AF-O06918-F1-model_v1.cif.gz
AF-O06918-F1-model_v1.pdb.gz
AF-O53113-F1-model_v1.cif.gz
AF-O53113-F1-model_v1.pdb.gz
AF-P0CL56-F1-model_v1.cif.gz
AF-P0CL56-F1-model_v1.pdb.gz
AF-P0CW37-F1-model_v1.cif.gz
AF-P0CW37-F1-model_v1.pdb.gz
AF-P0CW38-F1-model_v1.cif.gz
AF-P0CW38-F1-model_v1.pdb.gz
AF-P0CW39-F1-model_v1.cif.gz
AF-P0CW39-F1-model_v1.pdb.gz
AF-P0CW76-F1-model_v1.cif.gz
AF-P0CW76-F1-model_v1.pdb.gz
AF-P43409-F1-model_v1.cif.gz
AF-P43409-F1-model_v1.pdb.gz
AF-P54009-F1-model_v1.cif.gz
AF-P54009-F1-model_v1.pdb.gz
AF-P54010-F1-model_v1.cif.gz
AF-P54010-F1-model_v1.pdb.gz
AF-P54011-F1-model_v1.cif.gz
AF-P54011-F1-model_v1.pdb.gz
AF-P54012-F1-model_v1.cif.gz
AF-P54012-F1-model_v1.pdb.gz
AF-P54013-F1-model_v1.cif.gz
AF-P54013-F1-model_v1.pdb.gz
AF-P54014-F1-model_v1.cif.gz
AF-P54014-F1-model_v1.pdb.gz
AF-P54015-F1-model_v1.cif.gz
AF-P54015-F1-model_v1.pdb.gz
AF-P54016-F1-model_v1.cif.gz
AF-P54016-F1-model_v1.pdb.gz
AF-P54017-F1-m

In [5]:
# count number of pdb files
!ls data/*pdb.gz | wc -l

1773


In [6]:
## get uniprot ids from pdb filenames
files = os.listdir('data/')

uniprot_ids = []
for file in files:
    if file.startswith('AF-') and file.endswith('pdb.gz'):
        uniprot_id = file.split('-')[1]
        uniprot_ids.append(uniprot_id)
print(len(uniprot_ids))

1773


In [7]:
# https://www.uniprot.org/help/api_idmapping
# mapping uniprot id to pdb id
import urllib.parse
import urllib.request

url = 'https://www.uniprot.org/uploadlists/'

params = {
    'from': 'ACC+ID',
    'to': 'PDB_ID',
    'format': 'tab',
    'query': ' '.join(uniprot_ids)
}

data = urllib.parse.urlencode(params)
data = data.encode('utf-8')
req = urllib.request.Request(url, data)
with urllib.request.urlopen(req) as f:
    response = f.read()

res = response.decode('utf-8')

In [8]:
import io
id_mappings = pd.read_csv(io.StringIO(res), sep='\t')
id_mappings.columns = ['uniprot_id', 'pdb_id']
print(id_mappings.shape)
id_mappings.head()

(510, 2)


,uniprot_id,pdb_id
0,Q58256,6PEU
1,Q58346,1S3L
2,Q58346,1S3M
3,Q58346,1S3N
4,Q58346,2AHD


In [9]:
id_mappings.nunique()

uniprot_id    211
pdb_id        462
dtype: int64

In [10]:
id_mappings.drop_duplicates().shape

(510, 2)

In [11]:
pdb_ids = id_mappings['pdb_id'].unique()

In [12]:
with open('pdb_ids.txt', 'w') as out:
    out.write(','.join(list(pdb_ids)))

In [13]:
%%bash
# download from RCSB PDB

cd data
bash ../batch_download.sh -f ../pdb_ids.txt -p

Failed to download https://files.rcsb.org/download/4V4N.pdb.gz
Failed to download https://files.rcsb.org/download/4V7I.pdb.gz


In [14]:
!ls data/*.pdb.gz | wc -l

2233


## Connect to DocumentDB

In [16]:
from pymongo import MongoClient
from utils import get_secret

In [17]:
secrets = get_secret(stack_name)

In [18]:
uri = 'mongodb://{}:{}@{}:{}/?tls=true&tlsCAFile=rds-combined-ca-bundle.pem&replicaSet=rs0&readPreference=secondaryPreferred&retryWrites=false'\
    .format(secrets['username'], secrets['password'], secrets['host'], secrets['port'])

client = MongoClient(uri)

In [19]:
db = client['proteins'] # create a database
collection = db['proteins'] # create a collection

In [20]:
# collection.delete_many({}) # to delete all documents in this collection

In [21]:
collection.create_index('id', unique=True) # unique index in the collection

'id_1'

## Parse PDB files

In [17]:
!pip install biopython==1.79

     |████████████████████████████████| 2.3 MB 22.5 MB/s eta 0:00:01
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_latest_p36/bin/python -m pip install --upgrade pip' command.


In [22]:
from tqdm import tqdm
from Bio.PDB import PDBParser

import xpdb
from pdb_parse import parse_pdb_file_to_json_record

In [23]:
# PDB parser
pdb_parser = PDBParser(
    QUIET=True,
    PERMISSIVE=True,
    structure_builder=xpdb.SloppyStructureBuilder(),
)

## Parse and save data to DB

- for rcsb, use PDB-chain as id, add identifiers {uniprot_ids: [], pdb_ids: []}
- for AF, use `AF-Q58321` as id, add identifiers {uniprot_ids: [], pdb_ids: []}


In [24]:
import glob

In [25]:
# Parse PDB files from AlphaFold DB
for pdb_file in tqdm(glob.glob('data/AF-*.pdb.gz')):
    # AlphaFold structure
    id_ = '-'.join(os.path.basename(pdb_file).split('-')[:2])
    rec = parse_pdb_file_to_json_record(
        pdb_parser,
        pdb_file,
        id_
    )[0]
    
    uniprot_id = os.path.basename(pdb_file).split('-')[1]
    # look up pdb id
    pdb_ids = list(id_mappings.loc[id_mappings['uniprot_id']==uniprot_id, 'pdb_id'])
    identifiers = {
        'uniprot_ids': [uniprot_id],
        'pdb_ids': pdb_ids
    }
    rec['id'] = id_
    rec['identifiers'] = identifiers
    rec['is_AF'] = 1

    collection.insert_one(rec)

100%|██████████| 1773/1773 [01:49<00:00, 16.19it/s]


In [26]:
# Parse PDB files from RCSB
for pdb_file in tqdm(glob.glob('data/*.pdb.gz')):
    if not os.path.basename(pdb_file).startswith('AF-'):
        # RCSB structure
        pdb_id = os.path.basename(pdb_file).split('.')[0]
        recs = parse_pdb_file_to_json_record(
            pdb_parser,
            pdb_file,
            pdb_id
        )
        
        # look up uniprot_ids
        uniprot_ids = list(id_mappings.loc[id_mappings['pdb_id']==pdb_id, 'uniprot_id'])
        identifiers = {
            'uniprot_ids': uniprot_ids,
            'pdb_ids': [pdb_id]
        }
        for rec in recs:
            rec['id'] = rec['name']
            rec['identifiers'] = identifiers
            rec['is_AF'] = 0
        
        collection.insert_many(recs)

100%|██████████| 2233/2233 [01:29<00:00, 24.85it/s]


In [27]:
print('Number of protein chains in the DocumentDB:', 
      collection.count_documents({}))

Number of protein chains in the DocumentDB: 3151


## Load protein metadata from DocumentDB to split train/valid/test

In [28]:
from sklearn.model_selection import train_test_split

In [29]:
match = {"is_AF": {"$exists": True}}
project = {"y": "$is_AF"}

pipeline = [
    {"$match": match},
    {"$project": project},
]
# aggregation pipeline
cur = collection.aggregate(pipeline)
# retrieve documents from the DB cursor
docs = [doc for doc in cur]

In [30]:
# split train/valid/test
df = pd.DataFrame(docs)
print(df.shape)
df.head()

(3151, 2)


,_id,y
0,61391b70f03d70e1ee9122f3,1
1,61391b70f03d70e1ee9122f4,1
2,61391b70f03d70e1ee9122f5,1
3,61391b71f03d70e1ee9122f6,1
4,61391b71f03d70e1ee9122f7,1


In [31]:
# stratified split: full -> train/test
df_train, df_test = train_test_split(
    df, 
    test_size=0.2,
    stratify=df['y'], 
    random_state=42
)
print(df_train.shape, df_test.shape)

(2520, 2) (631, 2)


In [32]:
df_train['y'].mean(), df_test['y'].mean()

(0.5626984126984127, 0.5625990491283677)

In [33]:
# stratified split: train -> train/valid
df_train, df_valid = train_test_split(
    df_train, 
    test_size=0.2,
    stratify=df_train['y'], 
    random_state=42
)
print(df_train.shape, df_valid.shape, df_test.shape)

(2016, 2) (504, 2) (631, 2)


In [34]:
# Add split information into Documents
for split, df_split in zip(
    ['train', 'valid', 'test'],
    [df_train, df_valid, df_test]
):
    result = collection.update_many(
        {'_id': {'$in': df_split['_id'].tolist()}}, 
        {'$set': {'split': split}}
    )
    print('Number of documents modified:',result.modified_count)

Number of documents modified: 2016
Number of documents modified: 504
Number of documents modified: 631
